In [2]:
import numpy as np

# Hess 1 Tutorial

Import Objects

In [3]:
from emitter import CatalogEmitter, HealpixEmitter

In [4]:
from atmosphere import ExponentialAtmosphere

ImportError: cannot import name 'ExponentialAtmosphere' from 'atmosphere' (/home/anonymous/containers/apptainer/new_idea/build_0/atmosphere.py)

In [ ]:
from instrument import EffectiveApertureInstrument

Create specific realizations

# Basic usage:

In [ ]:
# Initialize submodels:
emitters = [emitter1, emitter2]

# Get them all into one model
model = Model(instrument, atmosphere, emitters)

Query the model, which sets up the scene to be rendered and the parameters to render with:

In [5]:
scene = model.query(observation)

NameError: name 'model' is not defined

Rendering a scene does not change the state of internal samplers:

In [ ]:
state_0 = scene.state.copy()
imgs = nsb2.render(scene)
print(scene.state == state_0)

To change the state of internal samplers, you need to call the step function:

In [ ]:
scene.step()
print(scene.state == state_0)

Display the scenes:

In [ ]:
nsb2.utils.visualize(imgs)

# Likelihood fitting (fixed state):
For fitting we use the render image to keep all the state vectors fixed and focus only on global parameters:

In [ ]:
import lmfit

# Gather all scene parameters into lmfit parameters
p = scene.parameters.to_lmfit()

# Creating residual function:
def residual(p):
    scene_instance = scene.realize(p)
    return np.mean(np.square(nsb2.render(scene_instance) - imgs))

# Minimize parameters using lmfit
mi = lmfit.minimize(residual, p)

# MCMC with fixed state

In [7]:
import lmfit

# Gather all scene parameters into lmfit parameters
p = scene.parameters.to_lmfit()

# Creating residual function:
def residual(p):
    scene_instance = scene.realize(p)
    return np.mean(np.square(nsb2.render(scene_instance) - imgs))

res = lmfit.minimize(residual, params=mi.params, method='emcee', nan_policy='omit', burn=200, steps=5000,
                     thin=5, nwalkers=30, progress=True)

NameError: name 'scene' is not defined

# MCMC with auxillary samplers
For our cases, some underlying models have uncertainties whose parameter space is determined by their own auxillary data (for example stellar catalog values). This can not be accounted for in a normal minimizer due to its randomness, but can be included in MCMC algorithms. We follow the same approach as previously, but now we also use .step() to sample the probability distribution of the underlying values.



In [ ]:
import lmfit

# Gather all scene parameters into lmfit parameters
p = scene.parameters.to_lmfit()

# Creating residual function:
def residual(p):
    scene.update(p)
    scene.step()
    return np.mean(np.square(nsb2.render(scene) - imgs))

res = lmfit.minimize(residual, params=mi.params, method='emcee', nan_policy='omit', burn=200, steps=5000,
                     thin=5, nwalkers=30, progress=True)

As before, the sample pipeline changes the internal state of the sampler.

In [1]:
WVL = 400
F32 = (2 * 2 * 49152 * WVL) + (49152*WVL*3) + (100000*WVL + 100000*WVL)
I32 = 960*32*32*10*10

In [2]:
F32*8/1e9

1.7410048

In [47]:
I32*8/1e9

0.786432